#### Decision Variables

-   $x_{pc} \in \{0, 1\}$:  
    = 1 if product $p$ is assigned to storage class $c$; 0 otherwise.

-   $y_{lc} \in \{0, 1\}$:  
    = 1 if location (or slot) $l$ is allocated to class $c$; 0 otherwise.

#### Parameters

-   $a_l$: area (or capacity) of location $l$.

-   $d_l$: "depth" or average travel-distance to location $l$.

-   $D_p$: total demand (or pick frequency) of product $p$.

-   $f, h$: fixed-cost and per-unit handling-cost coefficients.

-   (Optionally) $I_p^t, f_p$: if you enforce that in any time period $t$ the total picks assigned to class $c$ do not exceed its capacity.


#### Objective Function

$$
\min z = f \sum_c \sum_l a_l y_{lc} + 2h \sum_c \left[ \underbrace{\sum_l a_l d_l y_{lc}}_{\text{"avg. travel distance to class } c\text{"}} \times \underbrace{\sum_p D_p x_{pc}}_{\text{"total picks assigned to class } c\text{"}} \right]
$$

-   First term: total fixed cost of "opening" the area for each class.
-   Second: for each class $c$, avg. distance to that class $\times$ total picks assigned to that class, scaled by $2h$.


#### Constraints

##### 1. Assignment

$$
\sum_{c} x_{pc} = 1 \quad \forall p
$$

##### 2. Capacity (peak-load) — if enforced:

$$
\max_{t} \left[ \sum_{p} I_{p}^{t} f_{p} x_{pc} \right] \leq \sum_{l} a_{l} y_{lc} \quad \forall c
$$

##### 3. Location-uniqueness:

$$
\sum_{c} y_{lc} \leq 1 \quad \forall l
$$

##### 4. Binaries:

$$
x_{pc}, \, y_{lc} \in \{0, 1\}
$$


In Gurobi we can only work with linear or quadratic equation so we need to linearize the 2nd part of the objective function to make the model.


#### Import libraries


In [74]:
import gurobipy as gp
from gurobipy import GRB

In [75]:
#assuming parameter
n_products  = 20
n_classes   = 4
n_locations = 15

In [76]:
#assuming data
area      = {l: 10 + 2*l for l in range(n_locations)}    # a_l
depth     = {l: 5  + 1*l for l in range(n_locations)}    # d_l
demand    = {p: 100 + 5*p for p in range(n_products)}    # D_p
fixed_f  = 1000   # f
handle_h =    2   # h

# Global bounds for McCormick
# E[c] = ∑ a[l]*y[l,c], so:
E_min = 1e-6
E_max = sum(area.values())
# N[c] = ∑ D[p]*x[p,c], so:
N_min = 0.0
N_max = sum(demand.values())
# Ratio r[c] = T[c]/E[c] — estimate plausible bounds
r_min = 0.1
r_max = max(depth.values())


In [77]:
m = gp.Model("ClassFormation_PWL")

In [78]:
x  = m.addVars(n_products, n_classes, vtype=GRB.BINARY, name="x")
y  = m.addVars(n_locations, n_classes, vtype=GRB.BINARY, name="y")

E  = m.addVars(n_classes, lb=E_min, ub=E_max, name="E")   # total area per class
T  = m.addVars(n_classes, lb=0.0,  ub=E_max * r_max, name="T")   # weighted depth
N  = m.addVars(n_classes, lb=N_min, ub=N_max, name="N")   # total picks

r  = m.addVars(n_classes, lb=r_min, ub=r_max, name="r")   # ratio = T/E
zc = m.addVars(n_classes, lb=0.0,  ub=r_max * N_max, name="zc")  # zc = r * N

#### Constraints


In [79]:
for c in range(n_classes):
    m.addConstr(E[c] == gp.quicksum(area[l] * y[l, c] for l in range       (n_locations)), name=f"DefE_{c}")
    m.addConstr(T[c] == gp.quicksum(area[l] * depth[l] * y[l, c] for l in range(n_locations)),name=f"DefT_{c}")
    m.addConstr(N[c] == gp.quicksum(demand[p] * x[p, c] for p in range(n_products)),name=f"DefN_{c}")


In [80]:
for p in range(n_products):
    m.addConstr(gp.quicksum(x[p, c] for c in range(n_classes)) == 1,
                name=f"Assign_{p}")
for l in range(n_locations):
    m.addConstr(gp.quicksum(y[l, c] for c in range(n_classes)) <= 1,
                name=f"OneClassPerLoc_{l}")


In [81]:
for c in range(n_classes):
    m.addConstr(T[c] >= r_min * E[c] + E_min * r[c] - r_min * E_min, name=f"Mc1_T_{c}")
    m.addConstr(T[c] >= r_max * E[c] + E_max * r[c] - r_max * E_max, name=f"Mc2_T_{c}")
    m.addConstr(T[c] <= r_max * E[c] + E_min * r[c] - r_max * E_min, name=f"Mc3_T_{c}")
    m.addConstr(T[c] <= r_min * E[c] + E_max * r[c] - r_min * E_max, name=f"Mc4_T_{c}")
    m.addConstr(zc[c] >= r_min * N[c] + N_min * r[c] - r_min * N_min, name=f"Mc1_z_{c}")
    m.addConstr(zc[c] >= r_max * N[c] + N_max * r[c] - r_max * N_max, name=f"Mc2_z_{c}")
    m.addConstr(zc[c] <= r_max * N[c] + N_min * r[c] - r_max * N_min, name=f"Mc3_z_{c}")
    m.addConstr(zc[c] <= r_min * N[c] + N_max * r[c] - r_min * N_max, name=f"Mc4_z_{c}")

#### Objective function


In [82]:
#   f * Σ E[c]  +  2h * Σ zc[c]
obj_linear = fixed_f * gp.quicksum(E[c]  for c in range(n_classes))
obj_nl     = 2 * handle_h * gp.quicksum(zc[c] for c in range(n_classes))
m.setObjective(obj_linear + obj_nl, GRB.MINIMIZE)

In [83]:
m.write("Efficient formation of storage classes for warehouse storage location assignment.lp")
m.Params.TimeLimit = 300
m.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 79 rows, 160 columns and 440 nonzeros
Model fingerprint: 0x3f5ac5e2
Variable types: 20 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e-06, 3e+03]
  Objective range  [4e+00, 1e+03]
  Bounds range     [1e-06, 6e+04]
  RHS range        [1e-07, 6e+04]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 71 rows, 160 columns, 420 nonzeros
Variable types: 8 continuous, 152 integer (140 binary)

Root relaxation: objective 1.180000e+03, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [84]:
if m.status == GRB.OPTIMAL or m.solCount > 0:
    print(f"\nObjective Value: {m.ObjVal:.2f}\n")
    print("Product → Class assignments:")
    for p in range(n_products):
        assigned = False
        for c in range(n_classes):
            if x[p, c].X > 0.5:
                print(f"  Product {p} → Class {c}")
                assigned = True
                break
        if not assigned:
            print(f"  Product {p} → not assigned to any class")

    print("\nLocation → Class assignments:")
    for l in range(n_locations):
        assigned = False
        for c in range(n_classes):
            if y[l, c].X > 0.5:
                print(f"  Location {l} → Class {c}")
                assigned = True
                break
        if not assigned:
            print(f"  Location {l} → not assigned to any class")

else:
    print("No feasible solution found; status code:", m.status)



Objective Value: 1180.00

Product → Class assignments:
  Product 0 → Class 0
  Product 1 → Class 0
  Product 2 → Class 3
  Product 3 → Class 0
  Product 4 → Class 0
  Product 5 → Class 0
  Product 6 → Class 0
  Product 7 → Class 0
  Product 8 → Class 0
  Product 9 → Class 0
  Product 10 → Class 0
  Product 11 → Class 3
  Product 12 → Class 3
  Product 13 → Class 0
  Product 14 → Class 0
  Product 15 → Class 0
  Product 16 → Class 1
  Product 17 → Class 1
  Product 18 → Class 1
  Product 19 → Class 1

Location → Class assignments:
  Location 0 → not assigned to any class
  Location 1 → not assigned to any class
  Location 2 → not assigned to any class
  Location 3 → not assigned to any class
  Location 4 → not assigned to any class
  Location 5 → not assigned to any class
  Location 6 → not assigned to any class
  Location 7 → not assigned to any class
  Location 8 → not assigned to any class
  Location 9 → not assigned to any class
  Location 10 → not assigned to any class
  Location 